In [1]:
%tensorflow_version 2.x
!pip install -q tf-hub-nightly==0.8.0.dev201911110007
!pip install -q git+https://github.com/tensorflow/examples

TensorFlow 2.x selected.
     |████████████████████████████████| 92kB 4.8MB/s 


In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import cv2
import glob
import random
keras = tf.keras

In [3]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive/My\ Drive/RecycleSort/cifar100_dataset/

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive
/gdrive/My Drive/RecycleSort/cifar100_dataset


In [0]:
#Add images from training dataset
#labels- 0 is bottle, 1 is can
IMG_SIZE = 224
train_data = []
labels = []
files = glob.glob("train_data/*.PNG")
for myFile in files:
    #one_image = []
    image = cv2.imread(myFile)
    #Make Image Training Ready
    image = tf.cast(image, tf.float32)
    image = (image/127.5) - 1
    image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))
    train_data.append(image)
    #one_image.append(image)
    #Add the class label
    if myFile[11:14] == 'gla':
      label = 0
      labels.append(label)
      #one_image.append(label)
    elif myFile[11:14] == 'can':
      label = 1
      labels.append(label)
      #one_image.append(label)
    
      


In [0]:
#add images and labels from validation dataset. We split the dataset later in model.fit
files = glob.glob("val_data/*.PNG")
val_data = []
for file in files:
    one_image = []
    image = cv2.imread(file)
    #Make Image Training Ready
    image = tf.cast(image, tf.float32)
    image = (image/127.5) - 1
    image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))
    one_image.append(image)
    #Add the class label
    if file[9:12] == 'gla':
      label = 0
      one_image.append(label)
    elif file[9:12] == 'can':
      label = 1
      one_image.append(label)
    val_data.append(one_image)

In [97]:
print(len(train_data))
print(train_data[0].shape)
print(len(labels))

1000
(224, 224, 3)
1000


In [0]:
BATCH_SIZE = 32
SHUFFLE_BUFFER_SIZE = 100

In [0]:
IMG_SHAPE = (IMG_SIZE, IMG_SIZE, 3)

base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE, include_top=False, weights='imagenet')

In [109]:
base_model.trainable = False
base_model.summary()

Model: "mobilenetv2_1.00_224"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 225, 225, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 112, 112, 32) 864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 112, 112, 32) 128         Conv1[0][0]                      
_______________________________________________________________________________

In [0]:
prediction_layer = keras.layers.Dense(1)
prediction_batch = prediction_layer(feature_batch_average)
print(prediction_batch.shape)

In [0]:
model = tf.keras.Sequential([base_model, global_average_layer, prediction_layer])

In [0]:
base_learning_rate = 0.0001
model.compile(optimizer=tf.keras.optimizers.Adam(lr=base_learning_rate), loss='binary_crossentropy', metrics=['accuracy'])

In [0]:
model.summary()